# Task 
Given title and body of the question, predict the Question Tag (technical Domain). 

# Import libraries

In [4]:
import multiprocessing
import os

print(multiprocessing.cpu_count())
print(os.cpu_count())

4
4


In [5]:
!pip install -U spacy -qq

In [6]:
%%capture
!pip install wandb --upgrade

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#import sys
#sys.path.append('/content/drive/MyDrive/Custom_functions')
#sys.path.append('/home/harpreet/Insync/google_drive_shaannoor/Custom_functions')

In [9]:
# Import random function

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import  vocab

import wandb
import spacy
#import custom_preprocessor as cp

import random
from datetime import datetime
import numpy as np
from pathlib import Path
import pandas as pd
import joblib
from collections import Counter

from pathlib import Path

from sklearn.model_selection import train_test_split

In [10]:
spacy.__version__

'3.2.4'

In [11]:
#!python -m spacy download 'en_core_web_sm'

In [12]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [13]:
data_folder = Path('/content/drive/MyDrive/Data/NLP/')
#data_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/Data/NLP')

In [14]:
save_model_folder = Path('/content/drive/MyDrive/teaching_spring_2022/Models')
#save_model_folder = Path('/home/harpreet/Insync/harpreet24.utd@gmail.com/teaching_spring_2022/Models')

We will be using W&B for visualization.

In [15]:
# Login to W&B
wandb.login()

wandb: Currently logged in as: hsingh-utd (use `wandb login --relogin` to force relogin)


True

# **SubSet Stack Exchange**

This data is a subset of data from  Kaggle Competition (https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/data). The given dataset actually contains the different questions asked in the StackExchange website for various technical domains. We have fetched only those question that contains the top 10 individual tags. Each question can have only one tag. This means that this is a multi-class classification problem. These are the ten categories for tags in the data.

| Tag    | Language|
| -------| --------|
| 0      | c#      |
| 1      | jave    |
| 2      | php     |
| 3      | javascript|
| 4      | android |
| 5      | jquery  |
| 6      | c++     |
| 7      | python  |
| 8      | asp.net |

## Load the Data and create Train/Test/Valid Data Splits

In [16]:
#df = pd.read_csv(data_folder /'multiclass_hw.csv')

In [17]:
#df.head()

In [18]:
#df['combined_text'] = df.apply(lambda x: str(x['Title']) + ' ' + str(x['Body']), axis = 1 )

In [19]:
#cleaned_text = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=5000).transform(df['combined_text'].values)

In [20]:
#df['cleaned_text'] = cleaned_text

In [21]:
#df['Tag_Number_final'] = df['Tag_Number'].apply(lambda x: int(x[1]))

In [22]:
#df.head()

In [23]:
#df_final = df[['Title', 'Body','cleaned_text','Tags','Tag_Number_final']]

In [24]:
#df_final.head()

In [25]:
#df_final.to_csv(data_folder /'multiclass_hw_cleaned.csv')

In [26]:
df_hw = pd.read_csv(data_folder /'multiclass_hw_cleaned.csv')

In [27]:
df_hw.head()

,Unnamed: 0,Title,Body,cleaned_text,Tags,Tag_Number_final
0,0,detail disclosure indicator on UIButton,<p>Is there a simple way to place a detail dis...,detail disclosure indicator uibutton simple wa...,iphone,8
1,1,hello world fails to show up in emulator,<p>I followed Hello World tutorial exactly. E...,hello world fail emulator follow hello world t...,android,4
2,2,"Why is JSHint throwing a ""possible strict viol...",<p>Trying to validate some Javascript in JsHin...,jshint throw possible strict violation line tr...,javascript,3
3,3,Programmatically Make Bound Column Invisible,<p>I'm trying to make a data bound column invi...,programmatically bound column invisible try da...,asp.net,9
4,4,"More than one EditText - not getting focus, no...",<p>The home screen of my Android application h...,edittext get focus soft keyboard android home ...,android,4


In [28]:
X_final = df_hw['cleaned_text'].values

In [29]:
print(type(X_final), X_final.shape)

<class 'numpy.ndarray'> (188878,)


In [30]:
y_final = df_hw['Tag_Number_final'].values

In [31]:
from sklearn.model_selection import train_test_split

# Taking 80% of data as train and remaining 20% as test
X_train, x_valid_test, y_train, y_valid_test = train_test_split(X_final, y_final, test_size = 0.2, stratify = y_final)
X_valid, X_test, y_valid, y_test = train_test_split(x_valid_test, y_valid_test, test_size = 0.5, stratify = y_valid_test)

In [32]:
print(type(X_train))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Custom Dataset Class

In [33]:
class CustomDataset(torch.utils.data.Dataset):
    """IMDB dataset."""

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.X[idx]
        labels = self.y[idx]
        sample = (text, labels)
        
        return sample

In [34]:
trainset = CustomDataset(X_train,y_train)
validset = CustomDataset(X_valid,y_valid)
testset = CustomDataset(X_test,y_test)

In [35]:
trainset.__getitem__([11])

(array(['obtain browser scroll height java web application retrieve browser scroll height pass jsp backend code'],
       dtype=object), array([1]))

## Create Vocab

In [36]:
def create_vocab(dataset, min_freq):
  counter = Counter()
  for (text, _) in dataset:
    counter.update(str(text).split())
  my_vocab = vocab(counter, min_freq=min_freq)
  my_vocab.insert_token('<unk>', 0)
  my_vocab.set_default_index(0)
  return my_vocab

vocab should always be created based on trainset

In [37]:
se_vocab = create_vocab(trainset, min_freq = 5)

In [38]:
len(se_vocab)

84012

In [39]:
se_vocab.get_itos()[0:5]

['<unk>', 'graphic', 'overflow', 'problem', 'result']

In [40]:
se_vocab['abracadabra']

0

## Collate_fn for Data Loaders

In [41]:
# Creating a lambda function objects that will be used to get the indices of words from vocab
text_pipeline = lambda x: [se_vocab[token] for token in str(x).split()]
label_pipeline = lambda x: int(x)

In [42]:
'''
We know that input to the embedding layers are indices of words from the vocab.
The collate_batch() accepts batch of data and gets the indices of text from vocab and returns the same
We will include this collate_batch() in collat_fn attribute of DataLoader.
So it will create a batch of data containing indices of words and corresponding labels.
But for EmbeddingBag we need one more extra parameter, that is offset.
offsets determines the starting index position of each bag (sequence) in input.
'''
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list, label_list, offsets

## **Check Data Loader**



In [43]:
batch_size=2
check_loader= torch.utils.data.DataLoader(dataset=trainset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        collate_fn=collate_batch,
                                        num_workers=4)

In [44]:
for text, label, offsets in check_loader:
  print(label, text, offsets)
  break

tensor([4, 5]) tensor([  281,   367,  4231, 10469,   294,  1386,   177,   281,  4909,  4231,
          209,     0,  3516,  1386,    61,   294,   150,  1623,   318,  3274,
         4843,  1852,   762,  1317,   157,   472,  5499,   279,    61,   699,
          241,   508,     0,     0,  1517,   288,    16,  5499,   789,   318,
         1568,     0,    61,   508,     0, 74821,   289,  1056,     0,  4265,
         2367,   699,     0,   544,  1355,  1517,     0,   167,  4130,   296,
          272,  1892,   959,   508,  1056,   895,   543,   205,   841,   332,
           90,    16,  1187,     0]) tensor([ 0, 25])


# Model 


In [45]:
class MLPCustom(nn.Module):
  def __init__(self, vocab_size, hidden_sizes_list, dprobs_list, batchnorm_binary, output_dim, non_linearity):

    

    self.vocab_size = vocab_size
    

    self.hidden_sizes_list = hidden_sizes_list # hidden_sizes = [emb_dim, hidden_dim1, hidden_dim2, .....output_dim] # n + 1 elements
    self.dprobs_list = dprobs_list # dpropb =[prob1, prob2....probn] # n elements
    self.batchnorm_binary  = batchnorm_binary  # True or False
    self.output_dim = output_dim
    self.non_linearity = non_linearity
    

    super().__init__()

    # embedding_layer
    self.embedding_layer = nn.EmbeddingBag(self.vocab_size, self.hidden_sizes_list[0])

    # hidden layers
    self.hidden_layers = nn.ModuleList()

    # dropout layers
    self.dropout_layers = nn.ModuleList()

    # batchnorm layers
    self.batchnorm_layers = nn.ModuleList()

    for k in range(len(self.hidden_sizes_list)-1):
      self.hidden_layers.append(nn.Linear(self.hidden_sizes_list[k], self.hidden_sizes_list[k+1]))
      self.dropout_layers.append(nn.Dropout(p=self.dprobs_list[k]))

      if self.batchnorm_binary:
        self.batchnorm_layers.append(nn.BatchNorm1d(self.hidden_sizes_list[k+1], momentum=0.9))

    self.output_layer = nn.Linear(self.hidden_sizes_list[-1], self.output_dim)

   

  def forward(self, input, offsets):
    x = self.embedding_layer(input, offsets)
    for  k in range(len(self.hidden_sizes_list)-1):
      x =  self.non_linearity(self.hidden_layers[k](x))
      if self.batchnorm_binary:
        x = self.batchnorm_layers[k](x)
      x= self.dropout_layers[k](x)

    x = self.output_layer(x)
    # we are not using softmax function in the forward passs
    # nn.crossentropy loss (which we will use to define our loss) combines  nn.LogSoftmax() and nn.NLLLoss() in one single class
    return x 

# Training Functions

## Training Epoch 

In [46]:
def train(train_loader, model, optimizer, loss_function, log_batch, log_interval, grad_clipping, max_norm):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate.
  Output: final weights, bias, train loss, train accuracy
  """
  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_train
  global batch_ct_train

  # Training Loop loop
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode
  model.train()

  # Iterate on batches from the dataset using train_loader
  for inputs, targets, offsets in train_loader:
    
    # move inputs and outputs to GPUs
    inputs = inputs.to(device)
    targets = targets.to(device)
    offsets = offsets.to(device)

    # Forward pass
    output = model(inputs, offsets)
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    example_ct_train +=  len(targets)
    batch_ct_train += 1

    # set gradients to zero 
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Update parameters using their gradient
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)



  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)

  return train_loss, train_acc

## Validation/Test Epoch

In [47]:
def valid(loader, model, optimizer, loss_function, log_batch, log_interval):

  """ 
  Function for training the model and plotting the graph for train & valid loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and valid loss for each epoch.
  """

  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_valid
  global batch_ct_valid

  # Validation loop
  # Initialize train_loss at the he strat of the epoch
  running_valid_loss = 0
  running_valid_correct = 0
  
  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for inputs, targets, offsets in loader:
      
      # move inputs and outputs to GPUs
      inputs = inputs.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)

      # Forward pass
      output = model(inputs, offsets)
      loss = loss_function(output,targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      # count of images and batches
      example_ct_valid +=  len(targets)
      batch_ct_valid += 1

      # Add valid loss of a batch 
      running_valid_loss += loss.item()

      # Add correct count for each batch
      running_valid_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})


    # Calculate mean valid loss for the whole dataset for a particular epoch
    valid_loss = running_valid_loss/len(valid_loader)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()

    # Calculate accuracy for the whole dataset for a particular epoch
    valid_acc = running_valid_correct/len(valid_loader.dataset)
    
  return valid_loss, valid_acc

##  Model Training Loop


In [48]:
def train_loop(train_loader, valid_loader, model, loss_function, optimizer, epochs, device, patience, early_stopping,
               file_model):

  '''
  model: specify your model for training
  criterion: loss function 
  optimizer: optimizer like SGD , ADAM etc.
  train loader: function to carete batches for training data
  valid loader : function to create batches for valid data set
  file_model : specify file name for saving your model. This way we can upload the model weights from file. We will not to run model again.
  

  '''
  # Create lists to store train and valid loss at each epoch

  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []
  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False


  # Iterate for the given number of epochs
  for epoch in range(epochs):
    t0 = datetime.now()
    # Get train loss and accuracy for one epoch

    train_loss, train_acc = train(train_loader, model, optimizer, loss_function, 
                                  wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL,
                                  wandb.config.GRAD_CLIPPING, wandb.config.MAX_NORM)
    valid_loss, valid_acc = valid(valid_loader, model, optimizer, loss_function,
                                    wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss



    # Log the train and valid loss to W&B
    wandb.log({f"Train epoch Loss :": train_loss, f"Valid epoch Loss :": valid_loss })
    wandb.log({f"Train epoch Acc :": train_acc, f"Valid epoch Acc :": valid_acc})



    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


#Model Training

## **Meta data**



In [49]:
hyperparameters = dict(
    
    HIDDEN_SIZES_LIST = [300] + [200] + [100],  # 300 = embed_dim, 200- hidden_dim1 , 100 -hidden_dim2
    DPROBS_LIST = [0.5]*2,
    BATCHNORM_BINARY = True,
    
    
    VOCAB_SIZE = len(se_vocab),
    OUTPUT_DIM = 10,

    EPOCHS = 20,
    
    BATCH_SIZE = 256,
    LEARNING_RATE = 0.2,
    DATASET="IMDB",
    ARCHITECTUREe="Embed_2_hidden_layers",
    LOG_INTERVAL = 25,
    LOG_BATCH = True,
    FILE_MODEL = save_model_folder/'hw5_solution.pt',
    GRAD_CLIPPING = False,
    MAX_NORM = 0,
    MOMENTUM = 0,
    PATIENCE = 5,
    EARLY_STOPPING = True,
    SCHEDULER_FACTOR = 0,
    SCHEDULER_PATIENCE = 0,
    WEIGHT_DECAY = 0.000
   )

non_linearity = F.relu

## Initialize wandb

In [50]:
wandb.init(name = 'exp1', project = 'NLP_HW5_Solution', config = hyperparameters)

In [51]:
# add non_linearity to config file
wandb.config.NON_LINEARITY = non_linearity

## Specify Dataloader, Loss_function, Model, Optimizer, Weight Initialization

In [52]:
# Fix seed value
from datetime import datetime
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.BATCH_SIZE, shuffle = True, 
                                           collate_fn=collate_batch, num_workers = 4)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.BATCH_SIZE, shuffle = False, 
                                           collate_fn=collate_batch,  num_workers = 4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.BATCH_SIZE,   shuffle = False, 
                                          collate_fn=collate_batch,  num_workers = 4)

# cross entropy loss function
loss_function = nn.CrossEntropyLoss()

# use GPUs
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# model 
model_se = MLPCustom(wandb.config.VOCAB_SIZE, wandb.config.HIDDEN_SIZES_LIST, wandb.config.DPROBS_LIST, wandb.config.BATCHNORM_BINARY, 
           wandb.config.OUTPUT_DIM, non_linearity)
model_se.to(device)

def init_weights(m):
  if type(m) == nn.Linear:
      torch.nn.init.xavier_normal__(m.weight)
      torch.nn.init.zeros_(m.bias)
        
# apply initialization recursively  to all modules
#model_se.apply(init_weights)

# Intialize stochiastic gradient descent optimizer
#optimizer = torch.optim.Adam(model_imdb.parameters(), lr = wandb.config.LEARNING_RATE, weight_decay=wandb.config.WEIGHT_DECAY)
optimizer = torch.optim.SGD(model_se.parameters(), lr = wandb.config.LEARNING_RATE, weight_decay=wandb.config.WEIGHT_DECAY)

wandb.config.optimizer = optimizer

cuda:0


## Sanity Check
- Check the loss without any training. For Cross entropy the expected value will be log(number of classes)

In [53]:
# Fix seed value

SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

for inputs, targets, offsets in train_loader:
  
  # move inputs and outputs to GPUs
  inputs = inputs.to(device)
  targets = targets.to(device)
  offsets = offsets.to(device)
  model_se.eval()
  # Forward pass
  output = model_se(inputs, offsets)
  loss = loss_function(output, targets)
  print(f'Actual loss: {loss}')
  break

print(f'Expected Theoretical loss: {np.log(2)}')

Actual loss: 2.3230059146881104
Expected Theoretical loss: 0.6931471805599453


## Train Model and Save best model

In [54]:
wandb.watch(model_se, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [55]:
example_ct_train, batch_ct_train, example_ct_valid, batch_ct_valid = 0, 0, 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, valid_loader, model_se, loss_function, optimizer, 
                                                                                          wandb.config.EPOCHS, device,
                                                                                          wandb.config.PATIENCE, wandb.config.EARLY_STOPPING,
                                                                                          wandb.config.FILE_MODEL)

Validation loss has decreased (inf --> 0.700976). Saving Model...
Epoch : 1 / 20
Time to complete 1 is 0:00:15.432847
Train Loss:  0.9242 | Train Accuracy:  70.5173%
Valid Loss:  0.7010 | Valid Accuracy:  77.0913%

Validation loss has decreased (0.700976 --> 0.637378). Saving model...
Epoch : 2 / 20
Time to complete 2 is 0:00:16.878463
Train Loss:  0.7406 | Train Accuracy:  76.6608%
Valid Loss:  0.6374 | Valid Accuracy:  79.6696%

Validation loss has decreased (0.637378 --> 0.625537). Saving model...
Epoch : 3 / 20
Time to complete 3 is 0:00:15.748072
Train Loss:  0.6925 | Train Accuracy:  78.2055%
Valid Loss:  0.6255 | Valid Accuracy:  80.0191%

Validation loss has decreased (0.625537 --> 0.589936). Saving model...
Epoch : 4 / 20
Time to complete 4 is 0:00:15.834652
Train Loss:  0.6588 | Train Accuracy:  79.2226%
Valid Loss:  0.5899 | Valid Accuracy:  80.8185%

Validation loss has decreased (0.589936 --> 0.558046). Saving model...
Epoch : 5 / 20
Time to complete 5 is 0:00:15.934545
Tr

## Add Visulaization

### Add Loss plot

In [56]:
import matplotlib.pyplot as plt
# Plot the train loss and test loss per iteration
fig = plt.figure(0)
plt.plot(train_loss_history, label = 'train loss')
plt.plot(valid_loss_history, label = 'valid loss')
plt.legend()

# Log the plot to W&B
wandb.log({"train-test loss per epoch": fig})

/usr/local/lib/python3.7/dist-packages/plotly/matplotlylib/renderer.py:613: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



# **Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

## Function to get predictions

In [57]:
def get_acc_pred(data_loader, model):
  """ 
  Function to get predictions for a given test set and calculate accuracy.
  Input: Iterator to the test set.
  Output: Prections and Accuracy for test set.
  """
  model.eval()
  with torch.no_grad():
    # Array to store predicted labels
    predictions = torch.Tensor()
    predictions = predictions.to(device)

    # Array to store actual labels
    y = torch.Tensor()
    y = y.to(device)
    # Iterate over batches from test set
    for input, targets, offsets in data_loader:
      
      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)

      # Calculated the predicted labels
      output = model(input, offsets)

      # Choose the label with maximum probability
      indices = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, indices)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

    # Check for complete dataset if actual and predicted labels are same or not
    # Calculate accuracy
    acc = (predictions == y).float().mean()

  # Return array containing predictions and accuracy
  return predictions, acc
  

## Load saved model from file 

In [58]:
model_nn = MLPCustom(wandb.config.VOCAB_SIZE, wandb.config.HIDDEN_SIZES_LIST, wandb.config.DPROBS_LIST, wandb.config.BATCHNORM_BINARY, 
           wandb.config.OUTPUT_DIM, non_linearity)
model_nn.to(device)
model_nn.load_state_dict(torch.load(wandb.config.FILE_MODEL))

<All keys matched successfully>

In [59]:
print(wandb.config.FILE_MODEL)

/content/drive/MyDrive/teaching_spring_2022/Models/hw5_solution.pt


In [60]:
# Get the prediction and accuracy for the test dataset
predictions, acc_test = get_acc_pred(test_loader, model_nn)

In [61]:
# Print Accuracy for test dataset
print(acc_test * 100)

tensor(85.3187, device='cuda:0')


**We have obtained 85 % accuracy on test dataset.**


## **Visualizations on Predictions**

Now, we will make some visualizations for the predictions that we obtained.

We will construct a `confusion matrix` which will help us to visualize the performance of our classification model on the test dataset as we know the true values for the test data.

In [62]:
# Get an array containing actual labels
testing_labels = testset.y

In [77]:
df_hw.head()

,Unnamed: 0,Title,Body,cleaned_text,Tags,Tag_Number_final
0,0,detail disclosure indicator on UIButton,<p>Is there a simple way to place a detail dis...,detail disclosure indicator uibutton simple wa...,iphone,8
1,1,hello world fails to show up in emulator,<p>I followed Hello World tutorial exactly. E...,hello world fail emulator follow hello world t...,android,4
2,2,"Why is JSHint throwing a ""possible strict viol...",<p>Trying to validate some Javascript in JsHin...,jshint throw possible strict violation line tr...,javascript,3
3,3,Programmatically Make Bound Column Invisible,<p>I'm trying to make a data bound column invi...,programmatically bound column invisible try da...,asp.net,9
4,4,"More than one EditText - not getting focus, no...",<p>The home screen of my Android application h...,edittext get focus soft keyboard android home ...,android,4


In [90]:
tags = df_hw[['Tags', 'Tag_Number_final']].drop_duplicates().sort_values(by = ['Tag_Number_final'])

In [93]:
tags['Tags'].values

array(['c#', 'java', 'php', 'javascript', 'android', 'jquery', 'c++',
       'python', 'iphone', 'asp.net'], dtype=object)

In [82]:
tags

,Tags,Tag_Number_final
0,iphone,8
1,android,4
2,javascript,3
3,asp.net,9
5,c#,0
7,php,2
9,python,7
11,jquery,5
17,java,1
29,c++,6


In [97]:
testing_labels.shape

(18888,)

In [94]:
# Define the values for classes
classes = ['c#', 'java', 'php', 'javascript', 'android', 'jquery', 'c++',
       'python', 'iphone', 'asp.net']

# Log a confusion matrix to W&B
wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
                        probs = None,
                        y_true = testing_labels,
                        preds = predictions.to('cpu').numpy(),
                        class_names = classes)})


In [95]:
wandb.finish()

Train Batch Acc :,▁▂▃▃▂▄▃▃▅▆▆▅▇▅▇▅▆▅▅▅▇▅▇▆▇▇▄▃▇█▅▇▇▇▇▆▇▆▃▆
Train Batch Loss :,█▆▆▅▅▅▅▅▄▄▃▃▃▃▃▃▂▃▃▄▂▃▂▃▂▄▃▄▂▁▄▂▂▂▂▂▁▃▄▃
Train epoch Acc :,▁▄▅▅▆▆▆▆▇▇▇▇▇▇██████
Train epoch Loss :,█▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
Valid Batch Accuracy :,▃▃▁▃▁▁▃▅▅▄▃▃▄▂▄▇▇▃▄▆▆▄▅▅▇▄▇█▇▇▇▅▇▇▇▂▅▆▇▄
Valid Batch Loss :,▆▇█▅█▆▆▃▄▄▅▄▄█▄▂▂▅▅▂▄▆▄▂▃▅▃▂▃▁▂▃▂▃▁▇▃▂▂▅
Valid epoch Acc :,▁▃▃▄▅▅▅▆▆▇▇▇▇▇▇▇█▇██
Valid epoch Loss :,█▆▆▅▄▄▄▃▃▃▂▂▃▂▂▂▁▂▁▁
Train Batch Acc :,0.83203
Train Batch Loss :,0.56093
Train epoch Acc :,0.85101
